In [1]:
from pyspark.sql import SparkSession

import os
import sys

os.environ["JAVA_HOME"] = "C:\\Program Files\\Java\\jre1.8.0_441"
os.environ["SPARK_HOME"] = "C:\\Spark\\spark-3.5.5-bin-hadoop3" 
os.environ["HADOOP_HOME"] = "C:\\hadoop"
os.environ['PYSPARK_DRIVER_PYTHON_OPTS']= "notebook"
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
os.environ['PYSPARK_PYTHON'] = sys.executable

# 1. Create (or get) your Spark session
spark = SparkSession.builder \
    .appName("MyApp") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.executor.cores", "2") \
    .getOrCreate()

# 2. Read the CSV into a DataFrame
#    – header=True assumes the first row has column names
#    – inferSchema=True tells Spark to pick the right data types
df = spark.read.csv(
    "all_data.csv",   # or "s3a://bucket/all_data.csv" / local path
    header=True,
    inferSchema=True
)

# 3. Cache it in memory if you’ll be reusing it often
df = df.cache()

# 4. Inspect schema and row count to verify
df.printSchema()
print(f"Total rows: {df.count()}")

# 5. Separate features from label
all_cols   = df.columns
label_col  = all_cols[-1]    # last column as label
feature_cols = all_cols[:-1] # all but last

print(f"Label column: {label_col}")
print(f"Features ({len(feature_cols)} columns): {feature_cols}")

root
 |-- _c0: double (nullable = true)
 |-- _c1: double (nullable = true)
 |-- _c2: double (nullable = true)
 |-- _c3: double (nullable = true)
 |-- _c4: double (nullable = true)
 |-- _c5: double (nullable = true)
 |-- _c6: double (nullable = true)
 |-- _c7: double (nullable = true)
 |-- _c8: double (nullable = true)
 |-- _c9: double (nullable = true)
 |-- _c10: double (nullable = true)
 |-- _c11: double (nullable = true)
 |-- _c12: double (nullable = true)
 |-- _c13: double (nullable = true)
 |-- _c14: double (nullable = true)
 |-- _c15: double (nullable = true)
 |-- _c16: double (nullable = true)
 |-- _c17: double (nullable = true)
 |-- _c18: double (nullable = true)
 |-- _c19: double (nullable = true)
 |-- _c20: double (nullable = true)
 |-- _c21: double (nullable = true)
 |-- _c22: double (nullable = true)
 |-- _c23: double (nullable = true)
 |-- _c24: double (nullable = true)
 |-- _c25: double (nullable = true)
 |-- _c26: double (nullable = true)
 |-- _c27: double (nullable = tru

In [7]:
#from pyspark.sql import functions as F
#
## — make sure your raw df is materialized in cache:
#df = df.persist()  
#df.count()  # force materialization
#
## — compute mins & maxes in one job
#agg_exprs = (
#     [F.min(c).alias(f"{c}_min") for c in feature_cols]
#   + [F.max(c).alias(f"{c}_max") for c in feature_cols]
#)
#stats = df.agg(*agg_exprs).first().asDict()
#
## — build normalization expressions
#norm_exprs = []
#for c in feature_cols:
#    mn, mx = stats[f"{c}_min"], stats[f"{c}_max"]
#    if mx == mn:
#        expr = F.lit(0.0).alias(c)
#    else:
#        expr = ((F.col(c) - mn) / (mx - mn)).alias(c)
#    norm_exprs.append(expr)
#
## — preserve the label column
#norm_exprs.append(F.col(label_col))
#
## — one select does everything
#df_norm = df.select(*norm_exprs).persist()
#df_norm.count()  # now your data is normalized & cached

In [2]:
from pyspark.ml.feature import VectorAssembler, MinMaxScaler
from pyspark.ml.functions import vector_to_array
from pyspark.sql import functions as F

# — assume df is already cached and has been materialized (as in your snippet) —
#    and `feature_cols` is the list of numeric columns to normalize,
#    `label_col` is the name of the label column you want to preserve.

# 1. Assemble all feature columns into a single vector
assembler = VectorAssembler(
    inputCols=feature_cols,
    outputCol="features_vec"
)
df_vec = assembler.transform(df).cache()
df_vec.count()  # materialize assembler output

# 2. Fit a MinMaxScaler on that vector
scaler = MinMaxScaler(
    inputCol="features_vec",
    outputCol="scaled_vec"
)
scaler_model = scaler.fit(df_vec)  # one pass to compute per-dimension min/max

# 3. Transform the DataFrame to get “scaled_vec”
df_scaled = scaler_model.transform(df_vec).cache()
df_scaled.count()  # materialize scaling

# 4. Convert the “scaled_vec” column back into individual feature columns
#    Use vector_to_array to turn the Vector into an Array[Float], then pick each index.
df_scaled_arr = df_scaled.withColumn(
    "scaled_arr",
    vector_to_array("scaled_vec")
)

# 5. Build expressions to re-extract each normalized feature from the array
scaled_exprs = [
    F.col("scaled_arr")[i].alias(feature_cols[i])
    for i in range(len(feature_cols))
]

# 6. Select the normalized features + label column
df_norm = df_scaled_arr.select(
    *scaled_exprs,
    F.col(label_col)
).persist()

# 7. Materialize
df_norm.count()

# Now `df_norm` contains:
#   - each feature column replaced by its MinMax-scaled version in [0,1]
#   - the original label column untouched



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\mkaze\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\mkaze\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\mkaze\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.start()
  File "c:\Users\mkaze\anaconda3\Lib\site-pack

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\mkaze\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\mkaze\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\mkaze\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.start()
  File "c:\Users\mkaze\anaconda3\Lib\site-pack

AttributeError: _ARRAY_API not found

230

In [3]:
df_norm.show()

+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+-------------------+------------------+-------------------+-------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+----

In [4]:
import math
from collections import defaultdict
from operator import add
from pyspark.sql import Row
from pyspark.sql import functions as F

# --- Assumptions: ---
# spark: active SparkSession
# df_norm: DataFrame of normalized features (all values ∈ [0,1])
# feature_cols: list of feature column names (strings)

p = 4
width = 1.0 / p

# 2. Build df_binned: for each row of df_norm, compute each feature’s bin ID and the M-dimensional cube tuple
def to_binned_row(row):
    # row: pyspark.sql.Row containing feature_cols
    bin_ids = []
    for f in feature_cols:
        v = row[f]
        # compute bin ID = ceil(v / width), clamp to [1, p]
        b = int(math.ceil(v / width))
        b = min(max(b, 1), p)
        bin_ids.append(b)
    # Prepare a dict: "<feature>_bin": bin_ids[i], and "cube": tuple(bin_ids)
    data = {f"{feature_cols[i]}_bin": bin_ids[i] for i in range(len(feature_cols))}
    data["cube"] = tuple(bin_ids)
    return Row(**data)

# Convert df_norm → RDD of Rows with bin columns + cube
binned_rdd = df_norm.select(*feature_cols).rdd.map(to_binned_row)

# Convert back to DataFrame: columns ["<f1>_bin", "<f2>_bin", ..., "cube"]
df_binned = spark.createDataFrame(binned_rdd).cache()
df_binned.count()  # materialize cache

# 3. Build density grid (dg): map each cube → 1, then reduceByKey to get density per cube
cube_count_rdd = df_binned.rdd.map(lambda row: (row["cube"], 1))
dg_rdd = cube_count_rdd.reduceByKey(add)  # RDD[(cube: tuple, density: int)]
# Convert to DataFrame with schema ["cube", "density"]
dg = dg_rdd.map(lambda x: Row(cube=x[0], density=x[1])).toDF().cache()
dg.count()  # materialize cache
dg.orderBy(F.desc("density")).show(20, truncate=False)


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 22.0 failed 1 times, most recent failure: Lost task 0.0 in stage 22.0 (TID 17) (MoeinExpertBook executor driver): java.net.SocketException: Connection reset by peer: socket write error
	at java.net.SocketOutputStream.socketWrite0(Native Method)
	at java.net.SocketOutputStream.socketWrite(Unknown Source)
	at java.net.SocketOutputStream.write(Unknown Source)
	at java.io.BufferedOutputStream.write(Unknown Source)
	at java.io.DataOutputStream.write(Unknown Source)
	at java.io.FilterOutputStream.write(Unknown Source)
	at org.apache.spark.api.python.PythonRDD$.write$1(PythonRDD.scala:310)
	at org.apache.spark.api.python.PythonRDD$.$anonfun$writeIteratorToStream$1(PythonRDD.scala:322)
	at org.apache.spark.api.python.PythonRDD$.$anonfun$writeIteratorToStream$1$adapted(PythonRDD.scala:322)
	at scala.collecti

In [11]:
m_max = 10 
# ------------------------------------------------------------------
# 3.  JOIN BINS + DENSITY  →  df_dens  (each row: bins, density)
# ------------------------------------------------------------------
bin_cols = [f"{c}_bin" for c in feature_cols]

df_dens = (
    df_binned.join(dg.select("cube", "density"), on="cube")
             .select(*bin_cols, "density")
             .cache()
)
N = df_dens.count()         # total rows (broadcast later if large)

# ------------------------------------------------------------------
# 4.  RELEVANCE MI  I(feature ; density)
# ------------------------------------------------------------------
def rel_map(row):
    y = row["density"]
    for f in feature_cols:
        yield ((f, row[f"{f}_bin"], y), 1)

cont_rel_rdd = (
    df_dens.rdd.flatMap(rel_map)
               .reduceByKey(add)                   # (f, bin, dens) → count
               .cache()
)

# collect counts to driver
n_xy, n_x, n_y = defaultdict(int), defaultdict(int), defaultdict(int)
for (f, b, y), cnt in cont_rel_rdd.collect():
    n_xy[(f, b, y)] = cnt
    n_x[(f, b)]    += cnt
    n_y[y]         += cnt

mi_fd = {}
for f in feature_cols:
    I = 0.0
    for (ff, b, y), cnt in n_xy.items():
        if ff != f:
            continue
        pxy = cnt / N
        px  = n_x[(f, b)] / N
        py  = n_y[y]      / N
        I  += pxy * math.log(pxy / (px * py))
    mi_fd[f] = I

# relevance threshold Th  (Eq. 13)
Th = sum(mi_fd.values()) / len(mi_fd)

# ------------------------------------------------------------------
# 5.  GREEDY mRMRD SUBSPACE SELECTION
# ------------------------------------------------------------------
selected, candidates = [], feature_cols.copy()
mi_red = {}   # cache pairwise redundancy MI   (i,j) ↔ I(i;j)

while candidates and len(selected) < m_max:
    # --- choose next feature --------------------------------------
    if not selected:
        # first feature = highest relevance
        best = max(candidates, key=lambda f: mi_fd[f])
    else:
        s = selected[-1]   # only compute redundancy w.r.t. last selected

        # RDD: ((j, bin_j, bin_s), 1)
        def red_map(row):
            bs = row[f"{s}_bin"]
            for j in candidates:
                yield ((j, row[f"{j}_bin"], bs), 1)

        cont_rs_rdd = (
            df_binned.rdd.flatMap(red_map).reduceByKey(add)
        )

        # collect and update MI(j;s)
        n_xy_rs, n_x_rs, n_s_rs = defaultdict(int), defaultdict(int), defaultdict(int)
        for (j, b_j, b_s), cnt in cont_rs_rdd.collect():
            n_xy_rs[(j, b_j, b_s)] = cnt
            n_x_rs[(j, b_j)]      += cnt
            n_s_rs[b_s]           += cnt

        for j in candidates:
            if (j, s) in mi_red:
                continue
            I_js = 0.0
            for (jj, b_j, b_s), cnt in n_xy_rs.items():
                if jj != j:
                    continue
                pxy = cnt / N
                pj  = n_x_rs[(j, b_j)] / N
                ps  = n_s_rs[b_s]     / N
                I_js += pxy * math.log(pxy / (pj * ps))
            mi_red[(j, s)] = mi_red[(s, j)] = I_js

        # score each candidate
        best, best_score = None, float("-inf")
        for j in candidates:
            rel = mi_fd[j]
            if selected:                         # |S| ≥ 1
                red_avg = sum(mi_red[(j, t)] for t in selected) / len(selected)
            else:
                red_avg = 0.0
            score = rel - red_avg
            if score > best_score:
                best, best_score = j, score

    # --- early-stop check -----------------------------------------
    if mi_fd[best] < Th:
        print(f"Stopped (relevance {mi_fd[best]:.6f} < Th {Th:.6f})")
        break

    # --- finalise selection ---------------------------------------
    selected.append(best)
    candidates.remove(best)
    print(f"Selected {best:20s} | rel={mi_fd[best]:.6f} | |S|={len(selected)}")

print("\nFinal mRMRD subspace:", selected)

Selected _c463                | rel=-6.242094 | |S|=1
Selected _c169                | rel=-6.262135 | |S|=2
Selected _c528                | rel=-6.248209 | |S|=3
Selected _c499                | rel=-6.249723 | |S|=4
Selected _c40                 | rel=-6.264117 | |S|=5
Selected _c368                | rel=-6.254703 | |S|=6
Selected _c503                | rel=-6.259331 | |S|=7
Selected _c468                | rel=-6.255252 | |S|=8
Selected _c344                | rel=-6.259051 | |S|=9
Selected _c350                | rel=-6.268364 | |S|=10

Final mRMRD subspace: ['_c463', '_c169', '_c528', '_c499', '_c40', '_c368', '_c503', '_c468', '_c344', '_c350']


In [12]:
# ------------------------------------------------------------------
# LOF in the selected subspace (fixed join key)
# ------------------------------------------------------------------
from pyspark.ml.feature import VectorAssembler, BucketedRandomProjectionLSH
from pyspark.sql.window import Window
import pyspark.sql.functions as F
import math

k = MinPts = 50                      # same symbol as in the paper
m = len(selected)                    # subspace dimensionality

# 1. Projection to Fs
df_proj = (
    df_norm
      .select(*selected)
      .withColumn("id", F.monotonically_increasing_id())
      .cache()
)
df_proj.count()

# 2. Assemble into a feature vector
assembler = VectorAssembler(inputCols=selected, outputCol="features_vec")
df_vec = assembler.transform(df_proj).select("id", "features_vec").cache()
df_vec.count()

# 3. Approximate k-NN via LSH
lsh = BucketedRandomProjectionLSH(
    inputCol="features_vec",
    outputCol="hashes",
    bucketLength=math.sqrt(m) / 2.0
)
model = lsh.fit(df_vec)

max_dist = math.sqrt(m)                 # diagonal length of [0,1]^m
pairs = (
    model.approxSimilarityJoin(df_vec, df_vec, max_dist, distCol="dist")
         .select(F.col("datasetA.id").alias("pid"),
                 F.col("datasetB.id").alias("oid"),
                 "dist")
         .filter("pid < oid")              # keep one direction
)

pairs = pairs.unionByName(                # add the reverse direction
            pairs.selectExpr("oid as pid", "pid as oid", "dist")
        )

# 4. Keep the k nearest neighbours for each point
w = Window.partitionBy("pid").orderBy("dist")
knn = (
    pairs.withColumn("rn", F.row_number().over(w))
         .filter(F.col("rn") <= k)
         .select("pid", "oid", "dist")
         .cache()
)

# 5. k-distance of each oid
kdist = (
    knn.groupBy("oid")
        .agg(F.max("dist").alias("kdist"))
        .cache()
)

# 6. Reachability distance
rd = (
    knn.join(kdist, on="oid")
       .withColumn("reach_dist", F.greatest("dist", "kdist"))
       .cache()
)

# 7. Local reachability density
lrd = (
    rd.groupBy("pid")
       .agg((F.lit(k) / F.sum("reach_dist")).alias("lrd"))
       .cache()
)

# 8. LOF score
lof = (
    rd.join(lrd.select(F.col("pid").alias("oid"), "lrd"), on="oid")
       .groupBy("pid")
       .agg(F.avg("lrd").alias("avg_lrd_o"))
       .join(lrd, on="pid")
       .withColumn("LOF", F.col("avg_lrd_o") / F.col("lrd"))
       .cache()
)

# 9. Attach LOF scores back to the feature rows
outliers = (
    df_proj.join(
        lof.withColumnRenamed("pid", "id")   # ← rename key column
           .select("id", "LOF"),
        on="id"
    )
    .orderBy(F.desc("LOF"))
)

# Show top-N outliers
outliers.show(20, truncate=False)


+---+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+
|id |_c463              |_c169              |_c528              |_c499              |_c40               |_c368              |_c503              |_c468              |_c344              |_c350             |LOF               |
+---+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+
|86 |0.5140656685507183 |0.8788708825083671 |0.17182636255588898|0.5001397483320081 |0.46833061017522115|0.9999999999999999 |0.649250198726365  |0.3785172570840182 |0.45824650056249183|0.6908227075547878|1.0436874881372415|
|84 |0.46401605039478794|0.6941078034172978 |0.18840907804629572|0.6352024060566251 |0.40281398695957266